In [1]:
# File: notebooks/explore.ipynb (content for a Jupyter Notebook cell)
import os
import polars as pl
from sqlalchemy import create_engine
import requests

print("Running Jupyter Notebook for exploration...")

Running Jupyter Notebook for exploration...


In [9]:
# download the data from https://data.mobility.brussels/bike/api/counts/
url = "https://data.mobility.brussels/bike/api/counts/"
params = {
    'request':'devices',
    'outputFormat':'csv'
}

response = requests.get(url,params=params)

if response.status_code == 200:
    csv_content = response.text
    print(csv_content)
    with open('devices_list.csv', 'w') as file:
        file.write(csv_content)
else:
    print(f"Error: {response.status_code}")


Device name,Active,Road (nl),Road (fr),Road (en),Description (nl),Description (fr),Description (en),Lane schema,Basic schema,Detailed schema,Picture 1,Picture 2,Lon (WGS 84),Lat (WGS 84),X (Lb72),Y (Lb72)
CAT17,True,Veeweidekaai 11 - 1070 Anderlecht,Quai de Veeweyde 11 - 1070 Anderlecht,Quai de Veeweyde 11 - 1070 Anderlecht,Veeweidekaai 11 - 1070 Anderlecht,Quai de Veeweyde 11 - 1070 Anderlecht,Quai de Veeweyde 11 - 1070 Anderlecht,https://data.mobility.brussels/media/bike/count/CAT17_6b8e7a01-cc38-4d06-9a4d-3d4ece53a578.png,https://data.mobility.brussels/media/bike/count/CAT17_82c4bc7e-f1d0-43cc-9be5-8678af101c57.png,https://data.mobility.brussels/media/bike/count/CAT17_9be1e6e1-4c00-4730-b747-a6b158b85a5b.png,https://data.mobility.brussels/media/bike/count/CAT17_517a19f8-a282-4477-9375-fdbb513d4242.jpg,https://data.mobility.brussels/media/bike/count/CAT17_5b8ac796-d12b-48b0-8488-f7bd06847834.jpg,4.301991995919691,50.820621002525904,145295.77384021957,167800.59543291386
CB02411,True,V

In [10]:
import polars as pl
from io import StringIO
df = pl.read_csv(StringIO(csv_content))
print (df)


shape: (18, 17)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Device     ┆ Active ┆ Road (nl)  ┆ Road (fr) ┆ … ┆ Lon (WGS  ┆ Lat (WGS  ┆ X (Lb72)  ┆ Y (Lb72)  │
│ name       ┆ ---    ┆ ---        ┆ ---       ┆   ┆ 84)       ┆ 84)       ┆ ---       ┆ ---       │
│ ---        ┆ bool   ┆ str        ┆ str       ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64       │
│ str        ┆        ┆            ┆           ┆   ┆ f64       ┆ f64       ┆           ┆           │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CAT17      ┆ true   ┆ Veeweideka ┆ Quai de   ┆ … ┆ 4.301992  ┆ 50.820621 ┆ 145295.77 ┆ 167800.59 │
│            ┆        ┆ ai 11 -    ┆ Veeweyde  ┆   ┆           ┆           ┆ 384       ┆ 5433      │
│            ┆        ┆ 1070       ┆ 11 - 1070 ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ Anderle…   ┆ And…      ┆   ┆           ┆           

In [13]:
# ---Extract the device names ---
device_names = df["Device name"].to_list()
print(device_names)

['CAT17', 'CB02411', 'CB1101', 'CB1142', 'CB1143', 'CB1599', 'CB1699', 'CB2105', 'CEE016', 'CEK049', 'CEK18', 'CEK31', 'CEV011', 'CJE181', 'CJM90', 'CLW239', 'COM205', 'CVT387']


In [14]:
# ---Prepare to download the Bike Counts ---
import time

base_url = "https://data.mobility.brussels/bike/api/counts/"
start_date = "20190101"
end_date = "20241201"

for device in device_names:
    params = {
        'request': 'history',
        'featureID': device,
        'startDate': start_date,
        'endDate': end_date,
        'outputFormat': 'csv'
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        filename = f"history_{device}.csv"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"Saved {filename}")
    else:
        print(f"Failed for {device}: {response.status_code}")
    time.sleep(1)  # Be kind to the API



Saved history_CAT17.csv
Saved history_CB02411.csv
Saved history_CB1101.csv
Saved history_CB1142.csv
Saved history_CB1143.csv
Saved history_CB1599.csv
Saved history_CB1699.csv
Saved history_CB2105.csv
Saved history_CEE016.csv
Saved history_CEK049.csv
Saved history_CEK18.csv
Saved history_CEK31.csv
Saved history_CEV011.csv
Saved history_CJE181.csv
Saved history_CJM90.csv
Saved history_CLW239.csv
Saved history_COM205.csv
Saved history_CVT387.csv


In [16]:
# Combine all Data into o Single Dataframe
# Save raw data 
import glob
import re

def get_device_name(filename):
    # Remove path and extension, then split on '_'
    base = filename.split('/')[-1].replace('.csv', '')
    # Extract device name (everything after 'history_')
    return base.split('_', 1)[1]


# Collect all CSV files
csv_files = glob.glob("history_*.csv")
dfs = []

for f in csv_files:
    device_name = get_device_name(f)
    df = pl.read_csv(f)
    # Add device_name column
    df = df.with_columns(pl.lit(device_name).alias("device_name"))
    dfs.append(df)

# Combine all DataFrames
df_all = pl.concat(dfs)
print(df_all)

# Save to raw_data.csv
df_all.write_csv("../raw_data/raw_data.csv")
print(f'raw data saved')


shape: (3_059_094, 5)
┌────────────┬──────────┬───────┬───────────────┬─────────────┐
│ Date       ┆ Time gap ┆ Count ┆ Average speed ┆ device_name │
│ ---        ┆ ---      ┆ ---   ┆ ---           ┆ ---         │
│ str        ┆ i64      ┆ i64   ┆ i64           ┆ str         │
╞════════════╪══════════╪═══════╪═══════════════╪═════════════╡
│ 2020-11-23 ┆ 93       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 94       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 95       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 96       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-24 ┆ 1        ┆ 0     ┆ -1            ┆ CEK31       │
│ …          ┆ …        ┆ …     ┆ …             ┆ …           │
│ 2024-12-01 ┆ 92       ┆ 0     ┆ -1            ┆ CB1599      │
│ 2024-12-01 ┆ 93       ┆ 1     ┆ 32            ┆ CB1599      │
│ 2024-12-01 ┆ 94       ┆ 0     ┆ -1            ┆ CB1599      │
│ 2024-12-01 ┆ 95       ┆ 0     ┆ -1            ┆ CB1599      │
│ 2024-12-01 ┆ 96 

In [17]:
# --- Configuration (from Environment Variables) ---
DATABASE_HOST = os.getenv('DATABASE_HOST')
DATABASE_PORT = os.getenv('DATABASE_PORT')
DATABASE_USER = os.getenv('DATABASE_USER')
DATABASE_PASSWORD = os.getenv('DATABASE_PASSWORD')
DATABASE_NAME = os.getenv('DATABASE_NAME')

In [18]:
# Paths inside the Jupyter container due to volume mounts
RAW_DATA_CONTAINER_PATH = "/home/jovyan/raw_data/raw_data.csv"
OUTPUT_CSV_CONTAINER_PATH = "/home/jovyan/output/final_clean_data.csv"
PREPROCESSING_SCRIPT_CONTAINER_PATH = "/home/jovyan/scripts/preprocessing.py" # For importing if needed

In [36]:
# --- Load Unclean Data Directly ---
try:
    print(f"Loading unclean data from: {RAW_DATA_CONTAINER_PATH}")
    df_unclean = pl.read_csv(RAW_DATA_CONTAINER_PATH)
    df_unclean = df_unclean.rename({"device_name": "Device name"})
    print("Unclean Data Head:")
    print(df_unclean.head())
    print(df_unclean.columns)
except Exception as e:
    print(f"Error loading unclean data in notebook: {e}")
    df_unclean= pl.DataFrame() # Ensure df exists even on error


Loading unclean data from: /home/jovyan/raw_data/raw_data.csv
Unclean Data Head:
shape: (5, 5)
┌────────────┬──────────┬───────┬───────────────┬─────────────┐
│ Date       ┆ Time gap ┆ Count ┆ Average speed ┆ Device name │
│ ---        ┆ ---      ┆ ---   ┆ ---           ┆ ---         │
│ str        ┆ i64      ┆ i64   ┆ i64           ┆ str         │
╞════════════╪══════════╪═══════╪═══════════════╪═════════════╡
│ 2020-11-23 ┆ 93       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 94       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 95       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-23 ┆ 96       ┆ 0     ┆ -1            ┆ CEK31       │
│ 2020-11-24 ┆ 1        ┆ 0     ┆ -1            ┆ CEK31       │
└────────────┴──────────┴───────┴───────────────┴─────────────┘
['Date', 'Time gap', 'Count', 'Average speed', 'Device name']


In [33]:
#--- Load the Device List Data
DEVICE_LIST_PATH = "/home/jovyan/raw_data/devices_list.csv"
try:
    print(f"Attempting to load device list from: {DEVICE_LIST_PATH}")
    # Read CSV, Polars will infer types. We specify the join key to be String.
    df_devices = pl.read_csv(DEVICE_LIST_PATH, schema_overrides={"Device name": pl.String})

    # Validate required columns are present based on the provided headers
    required_device_cols = ["Device name", "Lon (WGS 84)", "Lat (WGS 84)", "X (Lb72)", "Y (Lb72)"]
    if not all(col in df_devices.columns for col in required_device_cols):
        raise ValueError(f"Device list CSV missing one of required columns: {required_device_cols}")

    print(f"Successfully loaded {df_devices.shape[0]} devices from the list.")
    print("Device List Data Head:")
    print(df_devices.head())
    print("Device List Schema:")
    print(df_devices.schema)
    print(df_devices.columns)

except Exception as e:
    print(f"Error loading device list: {e}")
    exit(1)


Attempting to load device list from: /home/jovyan/raw_data/devices_list.csv
Successfully loaded 18 devices from the list.
Device List Data Head:
shape: (5, 17)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Device     ┆ Active ┆ Road (nl)  ┆ Road (fr) ┆ … ┆ Lon (WGS  ┆ Lat (WGS  ┆ X (Lb72)  ┆ Y (Lb72)  │
│ name       ┆ ---    ┆ ---        ┆ ---       ┆   ┆ 84)       ┆ 84)       ┆ ---       ┆ ---       │
│ ---        ┆ bool   ┆ str        ┆ str       ┆   ┆ ---       ┆ ---       ┆ f64       ┆ f64       │
│ str        ┆        ┆            ┆           ┆   ┆ f64       ┆ f64       ┆           ┆           │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CAT17      ┆ true   ┆ Veeweideka ┆ Quai de   ┆ … ┆ 4.301992  ┆ 50.820621 ┆ 145295.77 ┆ 167800.59 │
│            ┆        ┆ ai 11 -    ┆ Veeweyde  ┆   ┆           ┆           ┆ 384       ┆ 5433      │
│            ┆        ┆ 1070    

In [37]:
# ---  Preprocess and Join Data with Polars ---
print("Starting Polars preprocessing and data enrichment (joining with geo-coordinates)...")
try:
    #  Basic cleaning on the main bike counts data
    # Use the now consistent "Device name" for operations if needed before join
    df_temp = df_unclean.with_columns(
        pl.col("Date").str.strptime(pl.Date, "%Y-%m-%d").alias("event_date"),
        # Calculate minutes from 'Time gap': (Time gap - 1) * 15 minutes
        ((pl.col("Time gap") - 1) * 15).alias("minutes_offset")
    ).with_columns(
        # Convert date to datetime and add the minute offset
        (pl.col("event_date").cast(pl.Datetime) + pl.duration(minutes=pl.col("minutes_offset")))
        .alias("timestamp")
    ).drop(["Date", "Time gap", "minutes_offset"]) # Drop original columns

    #  Perform the join operation to add spatial information
    # The 'on' column is now consistently "Device name"
    df_joined = df_temp.join(
        df_devices.select([
            "Device name", # This is now consistent
            "Lon (WGS 84)",
            "Lat (WGS 84)",
            "X (Lb72)",
            "Y (Lb72)"
        ]),
        on="Device name", # Join on the common column (now consistent)
        how="left"
    )

    #  Further cleaning, casting, and renaming on the joined data
    df_cleaned = df_joined.with_columns(
        pl.col("Count").cast(pl.Int64, strict=False), # Use 'Count' (capital C)
        pl.col("Average speed").cast(pl.Float64, strict=False), # Use 'Average speed' (capital A, space)
        # Explicitly cast and rename spatial columns
        pl.col("Lon (WGS 84)").cast(pl.Float64, strict=False).alias("longitude"),
        pl.col("Lat (WGS 84)").cast(pl.Float64, strict=False).alias("latitude"),
        pl.col("X (Lb72)").cast(pl.Float64, strict=False).alias("easting"),
        pl.col("Y (Lb72)").cast(pl.Float64, strict=False).alias("northing")
    ).drop_nulls(subset=[ # Drop rows if any of these crucial columns are null
        'timestamp', 'Count', 'Average speed', # Use original names for drop_nulls
        'longitude', 'latitude', 'easting', 'northing' # Use the new alias names here
    ]).filter(pl.col("Count") > 0) # Use 'Count' (capital C)

    #  Final column selection and renaming for output (database and CSV)
    df_cleaned = df_cleaned.select([
        "timestamp",
        pl.col("Device name").alias("device_name"), # Final rename for DB/CSV output
        pl.col("Count").alias("count"), # Rename for DB/CSV output
        pl.col("Average speed").alias("average_speed"), # Rename for DB/CSV output
        "longitude",
        "latitude",
        "easting",
        "northing"
    ])

    print(f"Successfully joined and cleaned data. New shape: {df_cleaned.shape}")
    print("Cleaned Data Head (with Latitude/Longitude/Easting/Northing):")
    print(df_cleaned.head())
    print("Cleaned Data Schema:")
    print(df_cleaned.schema)

except Exception as e:
    print(f"Error during Polars preprocessing and joining: {e}")
    # Print the full exception for better debugging
    import traceback
    traceback.print_exc()
    exit(1)

Starting Polars preprocessing and data enrichment (joining with geo-coordinates)...
Successfully joined and cleaned data. New shape: (2655797, 8)
Cleaned Data Head (with Latitude/Longitude/Easting/Northing):
shape: (5, 8)
┌─────────────┬─────────────┬───────┬────────────┬───────────┬───────────┬────────────┬────────────┐
│ timestamp   ┆ device_name ┆ count ┆ average_sp ┆ longitude ┆ latitude  ┆ easting    ┆ northing   │
│ ---         ┆ ---         ┆ ---   ┆ eed        ┆ ---       ┆ ---       ┆ ---        ┆ ---        │
│ datetime[μs ┆ str         ┆ i64   ┆ ---        ┆ f64       ┆ f64       ┆ f64        ┆ f64        │
│ ]           ┆             ┆       ┆ f64        ┆           ┆           ┆            ┆            │
╞═════════════╪═════════════╪═══════╪════════════╪═══════════╪═══════════╪════════════╪════════════╡
│ 2021-03-05  ┆ CEK31       ┆ 9     ┆ 18.0       ┆ 4.393038  ┆ 50.838182 ┆ 151710.663 ┆ 169752.309 │
│ 12:15:00    ┆             ┆       ┆            ┆           ┆         

In [ ]:
#--- save to database
#--- To Do
"""
if not df_cleaned.is_empty():
    conn_str = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
    try:
        print(f"Attempting to connect to PostgreSQL at {DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}...")
        engine = create_engine(conn_str)
        with engine.connect() as conn:
            df_cleaned.write_database(
                table_name="cleaned_data",
                connection=conn,
                if_exists="replace" # or "append", "fail"
            )
        print(f"Successfully wrote {df_cleaned.shape[0]} rows to 'cleaned_data' table in PostgreSQL.")
    except Exception as e:
        print(f"Error storing data to PostgreSQL: {e}")
        print("Please ensure your 'db' service is running and accessible and database credentials are correct.")
        import traceback
        traceback.print_exc() # Print full traceback for DB errors too
else:
    print("No cleaned data to store in PostgreSQL.")
    """


In [40]:
#--- Save Final Clean Data to CSV

if not df_cleaned.is_empty():
    try:
        df_cleaned.write_csv(OUTPUT_CSV_CONTAINER_PATH )
        print(f"Successfully saved cleaned data to CSV at: {OUTPUT_CSV_CONTAINER_PATH }")
        print(f"This file should now be available in your local './output' directory.")
    except Exception as e:
        print(f"Error saving final CSV: {e}")
else:
    print("No cleaned data to save to CSV.")

print("Preprocessing script finished.")



Successfully saved cleaned data to CSV at: /home/jovyan/output/final_clean_data.csv
This file should now be available in your local './output' directory.
Preprocessing script finished.


In [ ]:
# --- Data Exploration and Visualization ---
# Example: Use matplotlib/seaborn (assuming they are in notebooks/requirements.txt)
# import matplotlib.pyplot as plt
# import seaborn as sns
# if not df_final_csv.is_empty():
#     plt.figure(figsize=(10, 6))
#     sns.lineplot(data=df_final_csv, x="timestamp", y="value")
#     plt.title("Time Series of Cleaned Data")
#     plt.show()

print("\nNotebook exploration complete.")